In [1]:
import numpy as np
import scipy.linalg as lg
import matplotlib.pyplot as plt

In [2]:
def irs_optmization_signal_beamforming(system_parameters = [], *args):

    ##----- System parameters -----##

    ## Number of antennas at the receiver, number of antennas at the transmitter, number of IRSs elements, 
    ## number of IRSs, training overhead K defined as N*P, number of natural scatterers at the system. 

    Mt, Mr = system_parameters[0], system_parameters[1];
    N, P, K = system_parameters[2], system_parameters[3], system_parameters[4];
    L = P;
    
    ## Since the IRS is a 2D panel, it is necessary to divide the total number of elements following an 
    ## Uniform Rectangular Array (URA) array.
    Nx, Ny = int(N/4), int(4);
    
    
    
    
    
    ##----- Generating the channels -----##
    
    ## Angle of Departure (AoD), at the Transmitter (TX), and Angle of Arrival (AoA), at the Receiver (RX), respectively.
    theta_Tx = 2 * np.pi * np.random.randn(1,L) - np.pi
    theta_Rx = 2 * np.pi * np.random.randn(1,L) - np.pi

    ## Steering vectors TX-IRS and IRS-RX, respectively.
    B_tx  = 1/np.sqrt(1) * np.exp(1j * np.pi * np.linspace(0,Mt-1,Mt).reshape(Mt,1) * np.cos(theta_Tx));
    A_rx  = 1/np.sqrt(1) * np.exp(1j * np.pi * np.linspace(0,Mr-1,Mr).reshape(Mr,1) * np.cos(theta_Rx));

    ## AoA horizontal and AoA vertical at the IRS, respectively.
    theta_IRS_AoA_x  = 2 * np.pi * np.random.randn(1,P) - np.pi; 
    theta_IRS_AoA_y  = 2 * np.pi * np.random.randn(1,P) - np.pi; 

    ## AoD horizontal and AoA vertical at the IRS, respectively.
    theta_IRS_AoD_x  = 2 * np.pi * np.random.randn(1,P) - np.pi; 
    theta_IRS_AoD_y  = 2 * np.pi * np.random.randn(1,P) - np.pi; 
    
    ## Steering vectors AOA horizontal and vertical, respectively. 
    b_irs_x = 1/np.sqrt(1) * np.exp(1j*np.pi*(np.linspace(0, Nx-1,Nx).reshape(Nx,1) * np.cos(theta_IRS_AoA_x) * np.sin(theta_IRS_AoA_y)));
    b_irs_y = 1/np.sqrt(1) * np.exp(1j*np.pi*(np.linspace(0, Ny-1,Ny).reshape(Ny,1) * np.cos(theta_IRS_AoA_y)));
    B_irs   = np.kron(b_irs_y, b_irs_x); 

    ## Steering vectors AOD horizontal and vertical, respectively. 
    a_irs_x = 1/np.sqrt(1) * np.exp(1j*np.pi*(np.linspace(0, Nx-1,Nx).reshape(Nx,1) * np.cos(theta_IRS_AoD_x) * np.sin(theta_IRS_AoD_y)));
    a_irs_y = 1/np.sqrt(1) * np.exp(1j*np.pi*(np.linspace(0, Ny-1,Ny).reshape(Ny,1) * np.cos(theta_IRS_AoD_y)));
    A_irs   = np.kron(a_irs_y,a_irs_x); 

    ## Path gains
    alpha_pl =  1/np.sqrt(2) * (np.random.randn(L,1) + 1j * np.random.randn(L,1));
    beta_pl  =  1/np.sqrt(2) * (np.random.randn(L,1) + 1j * np.random.randn(L,1));

    ## Transmitted pilots signal are generated from the columns of a hadamard matrix, 
    ## since this way we obtain an orthogonal space of pilot signals. 
    x_pilots = lg.hadamard(K);
    x_pilots = x_pilots[0,0:Mt];
    
    
    
    
    
    ##----- Received pilot signals -----##

    ## Generating the AWGN component with zero mean and variance defined as 1/SNR_training
    ## for the noise vector.
    enery_per_symbol = np.mean(abs(x_pilots**2)); ## Energy symbol.
    var_noise = enery_per_symbol * 1/system_parameters[6]; ## variance of the AWGN component.
    noise_pilots = np.sqrt(var_noise/2) * (np.random.randn(P, Mr, K) + 1j*np.random.randn(P, Mr, K)); ## AWGN noise vector.

    ## IRS phase-shift matrix initialization as a DFT matrix.
    S = lg.dft(K);
    S = S[1:N+1, :]; ## It is necessary to obtain the phase-shift matrix with the dimensions of NxK.

    ## Pilot signals for each IRS P.
    
    ## DIMENSION ERROR ##
    y_pilots = np.zeros((P, Mr, K));
    for k in range(1,K+1):
        for p in range(1,P+1):
            y_pilots[p, :, k] = A_rx[:, p][:, np.newaxis] * alpha_pl[p] * np.transpose(A_irs[:, p]) * np.diag(S[:, k]) * B_irs[:, p] * beta_pl[p] * np.transpose(B_tx[:, p][:, np.newaxis]) * np.transpose(x_pilots) + noise_pilots[p, :, k];

    ## Known matrix C defined as S \Khatrirao X \Kronecker Imr. It is the same matrix for each IRS.
    K_bar = K/2;
    C     = np.zeros((K_bar, Mr, Mr*Mt*N));  

    ## New phase-shift matrix for the proposed scenario.
    ww = lg.dft(K);
    WW = np.zeros((K,K_bar));
    kk = 1;
    for k in range(1,K_bar+1):
        WW[:, k] = ww[:, kk+1] - ww[:, kk]; 
        kk = kk + 2;

    WW = WW[1:N, :]; ## N x K

    for k in range(1,K_bar+1):
        F_k = np.kron(x_pilots, np.eye(Mr));
        C[:, :, k] = np.kron(WW[:, k].transpose(),F_k);
                        
    return x_pilots
    ##return adr_irs, adr_no_irs, w_opt, q_opt, s_opt

In [ ]:
Mt, Mr = 4, 4 
N, P = 8, 2
K = N*Mt

SNR = 10**(20/10);
SNR_Train = 10**(25/10);Beta_IRS = 0.50;

system_parameters = [Mt, Mr, N, P, K, SNR, SNR_Train, Beta_IRS]

x_pilots = irs_optmization_signal_beamforming(system_parameters)
##[adr_irs, adr_no_irs, w_opt, q_opt, s_opt] = irs_optmization_signal_beamforming(system_parameters)